In [1]:
# =========================================
# 0. Import & cấu hình chung
# =========================================
import numpy as np
import pandas as pd

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    accuracy_score,
    f1_score,
    classification_report,
    confusion_matrix,
)
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.base import clone
from scipy.stats import randint, uniform

RANDOM_STATE = 42

# Thư mục chứa các file Elliptic (sửa lại cho đúng máy bạn)
DATA_DIR = r"D:\elliptic\Elliptic_Dataset"
TXS_FEATURES_FILE = "txs_features.csv"
TXS_CLASSES_FILE  = "txs_classes.csv"
TXS_EDGELIST_FILE = "txs_edgelist.csv"  # nếu cần cho GNN thì dùng thêm, ở đây chưa cần

In [2]:
# =========================================
# 1. Load dataset
# =========================================
import os

df_txs_features = pd.read_csv(os.path.join(DATA_DIR, TXS_FEATURES_FILE))
df_txs_classes  = pd.read_csv(os.path.join(DATA_DIR, TXS_CLASSES_FILE))

print("txs_features shape:", df_txs_features.shape)
print("txs_classes  shape:", df_txs_classes.shape)
print("txs_features columns:", df_txs_features.columns.tolist())
print("txs_classes  columns:", df_txs_classes.columns.tolist())


txs_features shape: (203769, 184)
txs_classes  shape: (203769, 2)
txs_features columns: ['txId', 'Time step', 'Local_feature_1', 'Local_feature_2', 'Local_feature_3', 'Local_feature_4', 'Local_feature_5', 'Local_feature_6', 'Local_feature_7', 'Local_feature_8', 'Local_feature_9', 'Local_feature_10', 'Local_feature_11', 'Local_feature_12', 'Local_feature_13', 'Local_feature_14', 'Local_feature_15', 'Local_feature_16', 'Local_feature_17', 'Local_feature_18', 'Local_feature_19', 'Local_feature_20', 'Local_feature_21', 'Local_feature_22', 'Local_feature_23', 'Local_feature_24', 'Local_feature_25', 'Local_feature_26', 'Local_feature_27', 'Local_feature_28', 'Local_feature_29', 'Local_feature_30', 'Local_feature_31', 'Local_feature_32', 'Local_feature_33', 'Local_feature_34', 'Local_feature_35', 'Local_feature_36', 'Local_feature_37', 'Local_feature_38', 'Local_feature_39', 'Local_feature_40', 'Local_feature_41', 'Local_feature_42', 'Local_feature_43', 'Local_feature_44', 'Local_feature_45',

In [3]:
# =========================================
# 2. Merge & xử lý label
# =========================================
# Giả định:
#   - df_txs_features có cột 'txId' và 'Time step' (hoặc 'time_step')
#   - df_txs_classes có cột 'txId' và 'class'
#   - class:
#       1 = licit
#       2 = illicit
#       3 = unknown
# Ta drop class=3, và tạo label nhị phân: 0=licit, 1=illicit

df = df_txs_features.merge(df_txs_classes, on="txId", how="left")

if "class" not in df.columns:
    raise ValueError("Không tìm thấy cột 'class' sau khi merge!")

print("\nPhân bố class ban đầu:")
print(df["class"].value_counts(dropna=False))

# Bỏ các giao dịch không có nhãn hoặc nhãn 'unknown' = 3
df = df.dropna(subset=["class"]).copy()
df["class"] = df["class"].astype(int)
df = df[df["class"] != 3].copy()

# Tạo label nhị phân: 0=licit, 1=illicit
df["label"] = (df["class"] == 2).astype(int)

LABEL_COL = "label"
print("\nPhân bố label sau khi bỏ unknown (0=licit,1=illicit):")
print(df[LABEL_COL].value_counts())


Phân bố class ban đầu:
class
3    157205
2     42019
1      4545
Name: count, dtype: int64

Phân bố label sau khi bỏ unknown (0=licit,1=illicit):
label
1    42019
0     4545
Name: count, dtype: int64


In [4]:
# =========================================
# 3. Chọn feature (loại id/time/label)
# =========================================
# Cột id / thời gian không dùng làm feature
possible_ts_cols = ["Time step", "time_step"]
ts_col = None
for c in possible_ts_cols:
    if c in df.columns:
        ts_col = c
        break

if ts_col is None:
    raise ValueError("Không tìm thấy cột time-step (ví dụ 'Time step' hoặc 'time_step')!")

drop_id_cols = ["txId", ts_col]
drop_label_cols = ["class", LABEL_COL]

cols_to_drop = [c for c in drop_id_cols + drop_label_cols if c in df.columns]

feature_cols = [c for c in df.columns if c not in cols_to_drop]

X_df = df[feature_cols].copy()
y = df[LABEL_COL].values
time_steps = df[ts_col].values

print("\nSố feature:", len(feature_cols))
print("Một vài feature đầu:", feature_cols[:10])

# Chỉ giữ các cột số
num_cols = X_df.select_dtypes(include=[np.number]).columns.tolist()
X_df = X_df[num_cols].copy()
print("\nSố feature numeric:", len(num_cols))


Số feature: 182
Một vài feature đầu: ['Local_feature_1', 'Local_feature_2', 'Local_feature_3', 'Local_feature_4', 'Local_feature_5', 'Local_feature_6', 'Local_feature_7', 'Local_feature_8', 'Local_feature_9', 'Local_feature_10']

Số feature numeric: 182


In [5]:
# =========================================
# 4. Chia train/val/test THEO TIME_STEP (30/10/10)
# =========================================
unique_ts = np.sort(df[ts_col].unique())
n_ts = len(unique_ts)
print("\nSố time-step khác nhau:", n_ts)
print("Các time-step đầu:", unique_ts[:10], "...", unique_ts[-10:])

# Nếu đúng Elliptic (49 time-step), ta dùng 30/10/9
# Nếu >= 50, dùng đúng 30/10/10
# Nếu ít hơn thì chia theo tỉ lệ 60/20/20
if n_ts == 49:
    train_ts = unique_ts[:30]
    val_ts   = unique_ts[30:40]
    test_ts  = unique_ts[40:]
elif n_ts >= 50:
    train_ts = unique_ts[:30]
    val_ts   = unique_ts[30:40]
    test_ts  = unique_ts[40:50]
else:
    # fallback: chia theo tỉ lệ tương đối
    idx_train_end = int(0.6 * n_ts)
    idx_val_end   = int(0.8 * n_ts)
    train_ts = unique_ts[:idx_train_end]
    val_ts   = unique_ts[idx_train_end:idx_val_end]
    test_ts  = unique_ts[idx_val_end:]

print("\nTime-step TRAIN:", train_ts[0], "->", train_ts[-1])
print("Time-step VAL  :", val_ts[0],   "->", val_ts[-1])
print("Time-step TEST :", test_ts[0],  "->", test_ts[-1])

train_mask = df[ts_col].isin(train_ts)
val_mask   = df[ts_col].isin(val_ts)
test_mask  = df[ts_col].isin(test_ts)

X_train = X_df[train_mask].values
y_train = y[train_mask]

X_val   = X_df[val_mask].values
y_val   = y[val_mask]

X_test  = X_df[test_mask].values
y_test  = y[test_mask]

print("\nKích thước:")
print("X_train:", X_train.shape, "y_train:", y_train.shape)
print("X_val  :", X_val.shape,   "y_val  :", y_val.shape)
print("X_test :", X_test.shape,  "y_test :", y_test.shape)

print("\nPhân bố nhãn train:")
print(pd.Series(y_train).value_counts())
print("\nPhân bố nhãn val:")
print(pd.Series(y_val).value_counts())
print("\nPhân bố nhãn test:")
print(pd.Series(y_test).value_counts())


Số time-step khác nhau: 49
Các time-step đầu: [ 1  2  3  4  5  6  7  8  9 10] ... [40 41 42 43 44 45 46 47 48 49]

Time-step TRAIN: 1 -> 30
Time-step VAL  : 31 -> 40
Time-step TEST : 41 -> 49

Kích thước:
X_train: (26905, 182) y_train: (26905,)
X_val  : (9686, 182) y_val  : (9686,)
X_test : (9973, 182) y_test : (9973,)

Phân bố nhãn train:
1    23951
0     2954
Name: count, dtype: int64

Phân bố nhãn val:
1    8619
0    1067
Name: count, dtype: int64

Phân bố nhãn test:
1    9449
0     524
Name: count, dtype: int64


In [6]:
# =========================================
# 5. Chuẩn hóa feature (fit trên TRAIN)
# =========================================
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled   = scaler.transform(X_val)
X_test_scaled  = scaler.transform(X_test)


In [7]:
# =========================================
# 6. AutoML đơn giản: random search trên VAL (macro-F1)
#    Không dùng k-fold
# =========================================
def sample_param(dist, rng):
    """Lấy 1 giá trị từ distribution hoặc list."""
    if hasattr(dist, "rvs"):
        # scipy distribution
        return dist.rvs(random_state=rng)
    # list / tuple / set ...
    dist = list(dist)
    return dist[rng.randint(0, len(dist))]

def random_search_single_model(
    name,
    base_estimator,
    param_dist,
    X_train, y_train,
    X_val,   y_val,
    n_iter=30,
    scoring="macro"
):
    """
    Random search đơn giản:
      - Mỗi iter: sample 1 bộ siêu tham số
      - Train trên TRAIN
      - Đánh giá trên VAL theo F1-macro
      - Chọn bộ tốt nhất, rồi train lại trên TRAIN+VAL với bộ đó
    """
    print(f"\n===== Random search cho {name} (không k-fold, dùng VAL) =====")
    rng = np.random.RandomState(RANDOM_STATE)
    best_f1 = -1.0
    best_params = None

    for i in range(n_iter):
        params = {k: sample_param(v, rng) for k, v in param_dist.items()}

        model = clone(base_estimator)
        model.set_params(**params)
        model.fit(X_train, y_train)

        y_val_pred = model.predict(X_val)
        # macro-F1 dùng để chọn model
        f1_macro = f1_score(y_val, y_val_pred, average="macro")

        print(f"Iter {i+1:02d}/{n_iter}: F1_macro(val) = {f1_macro:.4f}, params = {params}")

        if f1_macro > best_f1:
            best_f1 = f1_macro
            best_params = params

    print(f"\n>>> {name} – best F1_macro(val) = {best_f1:.4f}")
    print("Best params:", best_params)

    # Train lại trên TRAIN+VAL với best_params trước khi test
    X_train_full = np.vstack([X_train, X_val])
    y_train_full = np.concatenate([y_train, y_val])

    best_model = clone(base_estimator)
    best_model.set_params(**best_params)
    best_model.fit(X_train_full, y_train_full)

    return best_model

# Tính scale_pos_weight cho XGB/LGBM
n_pos = np.sum(y_train == 1)
n_neg = np.sum(y_train == 0)
scale_pos_weight = n_neg / n_pos if n_pos > 0 else 1.0
print("\nscale_pos_weight (train):", scale_pos_weight)


scale_pos_weight (train): 0.12333514258277316


In [8]:
# =========================================
# 7. Định nghĩa search space & chạy AutoML cho RF / XGB / LGBM
# =========================================

# 1) RandomForest
rf_base = RandomForestClassifier(random_state=RANDOM_STATE, n_jobs=-1)
rf_param_dist = {
    "n_estimators": randint(200, 600),
    "max_depth": randint(3, 30),
    "min_samples_split": randint(2, 50),
    "min_samples_leaf": randint(1, 20),
    "max_features": ["sqrt", "log2", None],
    "class_weight": [None, "balanced"]
}

best_rf = random_search_single_model(
    "RandomForest",
    rf_base,
    rf_param_dist,
    X_train_scaled, y_train,
    X_val_scaled,   y_val,
    n_iter=30
)

# 2) XGBoost
xgb_base = XGBClassifier(
    random_state=RANDOM_STATE,
    tree_method="hist",      # hoặc "gpu_hist" nếu chạy GPU
    eval_metric="logloss",
    use_label_encoder=False
)

xgb_param_dist = {
    "n_estimators": randint(200, 800),
    "max_depth": randint(3, 12),
    "learning_rate": uniform(0.01, 0.29),
    "subsample": uniform(0.6, 0.4),
    "colsample_bytree": uniform(0.6, 0.4),
    "min_child_weight": randint(1, 10),
    "gamma": uniform(0.0, 5.0),
    "scale_pos_weight": [scale_pos_weight]
}

best_xgb = random_search_single_model(
    "XGBoost",
    xgb_base,
    xgb_param_dist,
    X_train_scaled, y_train,
    X_val_scaled,   y_val,
    n_iter=30
)

# 3) LightGBM
lgbm_base = LGBMClassifier(
    random_state=RANDOM_STATE,
    objective="binary",
    n_jobs=-1
)

lgbm_param_dist = {
    "n_estimators": randint(200, 800),
    "max_depth": randint(-1, 15),
    "num_leaves": randint(16, 256),
    "learning_rate": uniform(0.01, 0.29),
    "subsample": uniform(0.6, 0.4),
    "colsample_bytree": uniform(0.6, 0.4),
    "min_child_samples": randint(5, 50),
    "scale_pos_weight": [scale_pos_weight]
}

best_lgbm = random_search_single_model(
    "LightGBM",
    lgbm_base,
    lgbm_param_dist,
    X_train_scaled, y_train,
    X_val_scaled,   y_val,
    n_iter=30
)



===== Random search cho RandomForest (không k-fold, dùng VAL) =====
Iter 01/30: F1_macro(val) = 0.9220, params = {'n_estimators': 302, 'max_depth': 22, 'min_samples_split': 30, 'min_samples_leaf': 15, 'max_features': None, 'class_weight': 'balanced'}
Iter 02/30: F1_macro(val) = 0.8852, params = {'n_estimators': 388, 'max_depth': 23, 'min_samples_split': 40, 'min_samples_leaf': 19, 'max_features': None, 'class_weight': None}
Iter 03/30: F1_macro(val) = 0.9181, params = {'n_estimators': 287, 'max_depth': 23, 'min_samples_split': 37, 'min_samples_leaf': 8, 'max_features': None, 'class_weight': 'balanced'}
Iter 04/30: F1_macro(val) = 0.8360, params = {'n_estimators': 508, 'max_depth': 4, 'min_samples_split': 25, 'min_samples_leaf': 12, 'max_features': 'log2', 'class_weight': 'balanced'}
Iter 05/30: F1_macro(val) = 0.9213, params = {'n_estimators': 585, 'max_depth': 23, 'min_samples_split': 34, 'min_samples_leaf': 12, 'max_features': 'log2', 'class_weight': 'balanced'}
Iter 06/30: F1_macro

d:\elliptic\venv1\Lib\site-packages\xgboost\core.py:158: UserWarning: [17:03:09] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Iter 01/30: F1_macro(val) = 0.8944, params = {'n_estimators': 302, 'max_depth': 6, 'learning_rate': np.float64(0.28570714885887566), 'subsample': np.float64(0.892797576724562), 'colsample_bytree': np.float64(0.8394633936788146), 'min_child_weight': 7, 'gamma': np.float64(2.229163764267956), 'scale_pos_weight': np.float64(0.12333514258277316)}


d:\elliptic\venv1\Lib\site-packages\xgboost\core.py:158: UserWarning: [17:03:10] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Iter 02/30: F1_macro(val) = 0.8571, params = {'n_estimators': 414, 'max_depth': 10, 'learning_rate': np.float64(0.10677549723031632), 'subsample': np.float64(0.6571467271687763), 'colsample_bytree': np.float64(0.8603553891795411), 'min_child_weight': 5, 'gamma': np.float64(4.8495492608099715), 'scale_pos_weight': np.float64(0.12333514258277316)}


...

[LightGBM] [Info] Number of positive: 23951, number of negative: 2954
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.010303 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 41712
[LightGBM] [Info] Number of data points in the train set: 26905, number of used features: 182
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.890206 -> initscore=2.092850
[LightGBM] [Info] Start training from score 2.092850
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain

d:\elliptic\venv1\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


[LightGBM] [Info] Number of positive: 23951, number of negative: 2954
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.009974 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 41712
[LightGBM] [Info] Number of data points in the train set: 26905, number of used features: 182
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.890206 -> initscore=2.092850
[LightGBM] [Info] Start training from score 2.092850
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain

d:\elliptic\venv1\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


Iter 30/30: F1_macro(val) = 0.9648, params = {'n_estimators': 383, 'max_depth': 9, 'num_leaves': 160, 'learning_rate': np.float64(0.22118274109743927), 'subsample': np.float64(0.7471132530877013), 'colsample_bytree': np.float64(0.8529223322374317), 'min_child_samples': 10, 'scale_pos_weight': np.float64(0.12333514258277316)}

>>> LightGBM – best F1_macro(val) = 0.9688
Best params: {'n_estimators': 430, 'max_depth': 12, 'num_leaves': 240, 'learning_rate': np.float64(0.08891835232154423), 'subsample': np.float64(0.8590760482165449), 'colsample_bytree': np.float64(0.6002081507981263), 'min_child_samples': 17, 'scale_pos_weight': np.float64(0.12333514258277316)}
[LightGBM] [Info] Number of positive: 32570, number of negative: 4021
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.013822 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 42202
[LightGBM] [Info] Number of data points in the train set: 36591, n

In [9]:
# =========================================
# 8. Đánh giá trên TEST (in Accuracy, F1 binary, micro, macro)
# =========================================
def evaluate_on_test(name, model, X_test, y_test):
    print(f"\n===== {name} trên TEST =====")
    y_pred = model.predict(X_test)

    acc      = accuracy_score(y_test, y_pred)
    f1_bin   = f1_score(y_test, y_pred)                 # pos_label=1 mặc định
    f1_micro = f1_score(y_test, y_pred, average="micro")
    f1_macro = f1_score(y_test, y_pred, average="macro")

    print("Accuracy :", acc)
    print("F1 (binary, pos_label=1):", f1_bin)
    print("F1 micro :", f1_micro)
    print("F1 macro :", f1_macro)

    print("\nclassification_report:")
    print(classification_report(y_test, y_pred, digits=4))

    print("Confusion matrix:")
    print(confusion_matrix(y_test, y_pred))

models = {
    "RandomForest": best_rf,
    "XGBoost":      best_xgb,
    "LightGBM":     best_lgbm,
}

for name, model in models.items():
    evaluate_on_test(name, model, X_test_scaled, y_test)


===== RandomForest trên TEST =====
Accuracy : 0.9764363782211972
F1 (binary, pos_label=1): 0.9877098478113069
F1 micro : 0.9764363782211972
F1 macro : 0.851430681481411

classification_report:
              precision    recall  f1-score   support

           0     0.9801    0.5630    0.7152       524
           1     0.9763    0.9994    0.9877      9449

    accuracy                         0.9764      9973
   macro avg     0.9782    0.7812    0.8514      9973
weighted avg     0.9765    0.9764    0.9734      9973

Confusion matrix:
[[ 295  229]
 [   6 9443]]

===== XGBoost trên TEST =====
Accuracy : 0.9655068685450717
F1 (binary, pos_label=1): 0.9818124140848049
F1 micro : 0.9655068685450717
F1 macro : 0.8242395403757358

classification_report:
              precision    recall  f1-score   support

           0     0.6772    0.6565    0.6667       524
           1     0.9810    0.9826    0.9818      9449

    accuracy                         0.9655      9973
   macro avg     0.8291   

d:\elliptic\venv1\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
